# Chrysalis SQLite Diff Visual

Compares two Diaphora/BinDiff-style SQLite databases from `databases/` and classifies functions as:

- `same`
- `patched`
- `added_in_patched`
- `removed_from_patched`

CSV outputs are written to `notebooks/tables/db_diff_reports/`.


In [ ]:
from __future__ import annotations

import hashlib
import sqlite3
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple

import pandas as pd

try:
    import plotly.express as px
    HAVE_PLOTLY = True
except Exception:
    HAVE_PLOTLY = False

ROOT = Path.cwd()
if not (ROOT / "databases").exists() and (ROOT.parent / "databases").exists():
    ROOT = ROOT.parent
DB_DIR = ROOT / "databases"
REPORT_DIR = ROOT / "notebooks" / "tables" / "db_diff_reports"
REPORT_DIR.mkdir(parents=True, exist_ok=True)

dbs = sorted([p for p in DB_DIR.glob("*.sqlite")])
if len(dbs) < 2:
    raise RuntimeError(f"Expected at least 2 sqlite DBs in {DB_DIR}, found {len(dbs)}")

def pick_pair(paths: List[Path]) -> Tuple[Path, Path]:
    legit = None
    patched = None
    for p in paths:
        n = p.name.lower()
        if "bluetoothservice" in n:
            legit = p
        if "patched" in n or "main_module" in n:
            patched = p
    if legit and patched:
        return legit, patched
    return paths[0], paths[1]

DB_LEGIT, DB_PATCHED = pick_pair(dbs)
print("LEGIT DB:", DB_LEGIT)
print("PATCHED DB:", DB_PATCHED)
print("REPORT DIR:", REPORT_DIR)


In [ ]:
@dataclass
class FnMeta:
    func_id: int
    name: str
    address: int
    size: int
    nodes: int
    edges: int


def load_function_meta(conn: sqlite3.Connection) -> Dict[int, FnMeta]:
    q = """
    SELECT id, COALESCE(name, ''), COALESCE(address, 0), COALESCE(size, 0), COALESCE(nodes, 0), COALESCE(edges, 0)
    FROM functions
    """
    out: Dict[int, FnMeta] = {}
    for fid, name, addr, size, nodes, edges in conn.execute(q):
        out[int(fid)] = FnMeta(
            func_id=int(fid),
            name=str(name),
            address=int(addr),
            size=int(size),
            nodes=int(nodes),
            edges=int(edges),
        )
    return out


def build_digests(db_path: Path) -> pd.DataFrame:
    conn = sqlite3.connect(str(db_path))
    try:
        meta = load_function_meta(conn)
        cur = conn.cursor()
        cur.execute(
            """
            SELECT func_id, address, COALESCE(mnemonic, ''), COALESCE(disasm, '')
            FROM instructions
            ORDER BY func_id, address
            """
        )

        rows = []
        last_fid = None
        m_hasher = hashlib.sha1()
        d_hasher = hashlib.sha1()
        inst_count = 0
        preview = []

        def flush(fid):
            nonlocal rows, m_hasher, d_hasher, inst_count, preview
            if fid is None or fid not in meta:
                return
            m = meta[fid]
            rows.append({
                "address": m.address,
                "name": m.name,
                "size": m.size,
                "nodes": m.nodes,
                "edges": m.edges,
                "inst_count": inst_count,
                "mnemonic_sha1": m_hasher.hexdigest(),
                "disasm_sha1": d_hasher.hexdigest(),
                "mnemonic_preview": " ".join(preview[:80]),
            })

        for fid, _addr, mnemonic, disasm in cur:
            fid = int(fid)
            if last_fid is None:
                last_fid = fid
            if fid != last_fid:
                flush(last_fid)
                last_fid = fid
                m_hasher = hashlib.sha1()
                d_hasher = hashlib.sha1()
                inst_count = 0
                preview = []

            m_tok = str(mnemonic)
            d_tok = str(disasm)
            m_hasher.update(m_tok.encode("utf-8", errors="ignore"))
            m_hasher.update(b"\x00")
            d_hasher.update(d_tok.encode("utf-8", errors="ignore"))
            d_hasher.update(b"\x00")
            inst_count += 1
            if len(preview) < 80:
                preview.append(m_tok)

        flush(last_fid)
        df = pd.DataFrame(rows)
        if not df.empty:
            df["address_hex"] = df["address"].map(lambda x: f"0x{x:08X}")
            df = df.sort_values(["address", "name"]).reset_index(drop=True)
        return df
    finally:
        conn.close()


df_legit = build_digests(DB_LEGIT)
df_patched = build_digests(DB_PATCHED)
print("Functions (legit):", len(df_legit))
print("Functions (patched):", len(df_patched))
if df_legit.empty or df_patched.empty:
    raise RuntimeError("Failed to compute function digests from one or both DBs")


In [ ]:
cmp_cols = [
    "address", "name", "size", "nodes", "edges", "inst_count", "mnemonic_sha1", "disasm_sha1", "mnemonic_preview"
]
L = df_legit[cmp_cols].copy()
P = df_patched[cmp_cols].copy()
merged = L.merge(P, on="address", how="outer", suffixes=("_legit", "_patched"), indicator=True)

def classify(row):
    if row["_merge"] == "left_only":
        return "removed_from_patched"
    if row["_merge"] == "right_only":
        return "added_in_patched"
    if row["mnemonic_sha1_legit"] == row["mnemonic_sha1_patched"] and row["disasm_sha1_legit"] == row["disasm_sha1_patched"]:
        return "same"
    return "patched"

merged["classification"] = merged.apply(classify, axis=1)
patched_df = merged[merged["classification"] == "patched"].copy()
if not patched_df.empty:
    patched_df["inst_delta"] = (patched_df["inst_count_patched"].fillna(0) - patched_df["inst_count_legit"].fillna(0)).abs()
    patched_df["size_delta"] = (patched_df["size_patched"].fillna(0) - patched_df["size_legit"].fillna(0)).abs()

summary = merged.groupby("classification", dropna=False).size().reset_index(name="count").sort_values("count", ascending=False)
summary


In [ ]:
summary.to_csv(REPORT_DIR / "summary.csv", index=False)
merged.to_csv(REPORT_DIR / "all_functions_classified.csv", index=False)
merged[merged["classification"] == "same"].to_csv(REPORT_DIR / "same_functions.csv", index=False)
merged[merged["classification"] == "added_in_patched"].to_csv(REPORT_DIR / "added_in_patched.csv", index=False)
merged[merged["classification"] == "removed_from_patched"].to_csv(REPORT_DIR / "removed_from_patched.csv", index=False)
if not patched_df.empty:
    patched_df.sort_values(["inst_delta", "size_delta"], ascending=False).to_csv(REPORT_DIR / "patched_functions.csv", index=False)
    patched_df[["address", "name_legit", "name_patched", "mnemonic_preview_legit", "mnemonic_preview_patched"]].head(200).to_csv(REPORT_DIR / "patched_preview_mnemonics.csv", index=False)

print("Wrote reports to", REPORT_DIR)
for p in sorted(REPORT_DIR.glob("*.csv")):
    print(" -", p.name)


In [ ]:
if HAVE_PLOTLY:
    fig = px.bar(summary, x="classification", y="count", title="Function Classification Summary")
    fig.show()
else:
    print("Plotly not installed; skipping chart")

if not patched_df.empty:
    cols = [
        "address", "name_legit", "name_patched",
        "inst_count_legit", "inst_count_patched",
        "size_legit", "size_patched",
        "inst_delta", "size_delta"
    ]
    display(patched_df.sort_values(["inst_delta", "size_delta"], ascending=False)[cols].head(50))
else:
    print("No patched functions detected")
